In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

### Local MCP Server

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/mcp_server.py"],
            }
    }

)

In [ ]:
from langchain.chat_models import init_chat_model
model_gemini_flash = init_chat_model("gemini-2.5-flash", model_provider="google_genai", timeout=30, temperature=0)
model_llama_groq = init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=30, temperature=0)

In [ ]:
# get tools
tools = await client.get_tools()
# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [ ]:
print(prompt)

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

agent = create_agent(
    model=model_llama_groq,
    tools=tools,
    system_prompt=prompt
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [ ]:
for msg in response["messages"]:
    msg.pretty_print()

### Online MCP

In [ ]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [ ]:
agent = create_agent(
    model=model_gemini_flash,
    tools=tools,
    system_prompt="If the user asks about time, repond only with the time and timezone information. No additional natural language text in the response"
)

In [ ]:
question = HumanMessage("What time is it?")

response = await agent.ainvoke({"messages":[question]})



In [ ]:
for msg in response["messages"]:
    msg.pretty_print()

### Travel agent

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model_gemini_flash,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel agent. No follow up questions."
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "3"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct flight from San Francisco to Tokyo on 30th December 2025")]},
    config
    )

In [ ]:
for msg in response["messages"]:
    msg.pretty_print()